#[문제]
- law_2.docx 파일을 읽고, chroma 저장  
- LLM질문 -> 답변  
- 전세사기피해에 관한 법률 질문만 받기.
- 이 외의 질문은 '답변을 할 수 없습니다.'

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx') #인스턴스 생성 #문서 확인

text_splitter = RecursiveCharacterTextSplitter(  #분할을 하기위한 설정
    chunk_size=1500,
    chunk_overlap=200,
) #나중에 여러번의 질문을 통해 정확도를 확인
document_list = loader.load_and_split(text_splitter=text_splitter) #파일을 갖고 파라미터에 설정된 읽고 분할

## 2. 임베딩 -> 벡터 데이터베이스에 저장 
## 2.1. 환경변수 읽어오기
load_dotenv() #잘 읽어왔다는 True

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large') #성능 좋은 유료  # 위에서 OPENAI_API_KEY 읽었기 때문에 

# # ## 2.3. 벡터 데이터베이스에 저장 ########한번 실행했기 때문에
##[방법 1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

## 로컬에 저장된 임베딩 데이터 가져오기

database =Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)


## 3. 질문이 있으면, 벡터 데이터베이스(vector_store)에서 유사도 검색

## 3.1. 사용자 질문
# query = '오늘 서울 날씨는?'
query = '전세사기 피해자는 누구인가요? '

## 3.2. 벡터 데이터베이스에서 유사도 검색 (점수 포함)
retrieved_docs = database.similarity_search(query=query, k=2)

## 문서 객체에서 문자열만 추출



## 3.3 문서 객체 -> 하나의 문자열
## 리스트 for 문
context = '🤷‍♀️🤷‍♀️\n\n'.join([doc.page_content for doc in retrieved_docs])
context


## 4.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

## 4.1. 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기 피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기 피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=context,
    query=query,
)

## 4.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)  
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message.content

'"전세사기피해자"는 전세사기로 인한 피해를 입은 임차인으로, 제3조의 요건을 모두 갖춘 후 전세사기피해지원위원회의 심의 및 의결을 거쳐 국토교통부장관이 결정한 임차인을 말합니다.'